In [3]:
import pandas as pd
import os

df = pd.read_csv(r"C:\Users\gaeta\Documents\YNOV\master1\open_data\visualisation\df_ti_pe_complet_calculmeter.csv", sep=',')
df = df.reset_index(drop=True)

print(df.head(15))

print(df.dtypes)

   date_mutation  valeur_fonciere code_voie  code_postal  code_commune  \
0     2023-11-29           553500      0811        31770           149   
1     2023-11-15           286110      B058        31400           555   
2     2023-11-15           406414      B055        31590           573   
3     2023-11-15                1      B053        31790           490   
4     2023-11-22                1      1580        31600           395   
5     2023-11-29           424000      0802        31770           149   
6     2023-11-15          1954000      B063        31300           555   
7     2023-11-23           330000      2975        31200           555   
8     2023-11-27           312480      0693        31120           433   
9     2023-11-15            65500      B032        31340            73   
10    2023-11-15           520000      B045        31520           446   
11    2023-11-15             1200      B017        31360            50   
12    2023-11-15             1000     

In [4]:
df_preformatted = df

# PE= float -> transformation en int
# df_preformatted['PE'] = df_preformatted['PE'].str.replace(',', '.').astype('float').fillna(0).astype('int')
df_preformatted['PE'] = df_preformatted['PE'].fillna(0).astype('int')

# tt_surf_carrez = float -> transformation en int
# df_preformatted['tt_surf_carez'] = df_preformatted['tt_surf_carez'].fillna(0).astype('int')

# val_foncière = supprimer les valeurs négatives
# df_preformatted = df_preformatted.loc[df_preformatted['valeur_fonciere'] > 0]

# df_preformatted['year_date_mutation'] = pd.to_datetime(df_preformatted['date_mutation'])
# df_preformatted['year_date_mutation'] = df_preformatted['year_date_mutation'].dt.year

##### ajout evolution PE en % SUR 5 ANS

In [5]:
df_years_PE = df[['date_mutation', 'code_commune', 'PE']]

df_years_PE['year_date_mutation'] = pd.to_datetime(df_years_PE['date_mutation'])
df_years_PE['year_date_mutation'] = df_years_PE['year_date_mutation'].dt.year
df_years_PE = df_years_PE[['year_date_mutation', 'code_commune', 'PE']]
print(df_years_PE)

sum_pe = df_years_PE.groupby(['year_date_mutation', 'code_commune'])['PE'].sum().reset_index()
sum_pe = sum_pe.rename(columns={'PE': 'sum_pe_by_commune_year'})
print(sum_pe)

df_2019 = sum_pe[sum_pe['year_date_mutation'] == 2019][['code_commune', 'sum_pe_by_commune_year']]
df_2023 = sum_pe[sum_pe['year_date_mutation'] == 2023][['code_commune', 'sum_pe_by_commune_year']]

df_2019 = df_2019.rename(columns={'sum_pe_by_commune_year': 'sum_2019'})
df_2023 = df_2023.rename(columns={'sum_pe_by_commune_year': 'sum_2023'})

merged_df = pd.merge(df_2019, df_2023, on='code_commune', how='inner')
merged_df['%evolution_PE_2019_2023'] = ((merged_df['sum_2023'] - merged_df['sum_2019']) / merged_df['sum_2019']) * 100
merged_df = merged_df[['code_commune', '%evolution_PE_2019_2023']]
# merged_df['%evolution_PE_2019_2023'] =merged_df['%evolution_PE_2019_2023'].astype('float')
# merged_df['%evolution_PE_2019_2023'] =merged_df['%evolution_PE_2019_2023'].replace([float('inf'), -float('inf')], 0)
# merged_df['%evolution_PE_2019_2023'] =merged_df['%evolution_PE_2019_2023'].astype('float').fillna(0).astype('int64')
# merged_df['%evolution_PE_2019_2023']=merged_df['%evolution_PE_2019_2023'].round().replace([float('inf'), -float('inf')], 0).astype('int64')
print(merged_df.dtypes)



df_preformatted = pd.merge(df_preformatted, merged_df, on='code_commune', how='left')
df_preformatted['%evolution_PE_2019_2023'] = df_preformatted['%evolution_PE_2019_2023'].fillna(0).round().replace([float('inf'), -float('inf')], 0).astype('int64')
print(df_preformatted)

        year_date_mutation  code_commune     PE
0                     2023           149  17008
1                     2023           555   8792
2                     2023           573  12489
3                     2023           490      0
4                     2023           395      0
...                    ...           ...    ...
169043                2019           555   5485
169044                2019           555  16030
169045                2019           555   1200
169046                2019           555   2075
169047                2019           555  17074

[169048 rows x 3 columns]
      year_date_mutation  code_commune  sum_pe_by_commune_year
0                   2019             1                  133763
1                   2019             2                   99602
2                   2019             3                  299799
3                   2019             4                  844873
4                   2019             5                  110674
...                

C:\Users\gaeta\AppData\Local\Temp\ipykernel_12164\3647388987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_years_PE['year_date_mutation'] = pd.to_datetime(df_years_PE['date_mutation'])
C:\Users\gaeta\AppData\Local\Temp\ipykernel_12164\3647388987.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_years_PE['year_date_mutation'] = df_years_PE['year_date_mutation'].dt.year


#### colonne qui indique les types de maison appartement dependance

In [6]:
def create_combined_property_type(row):
    categories = []
    if row['appartement']:
        categories.append('appartement')
    if row['maison']:
        categories.append('maison')
    if row['local_indus_com']:
        categories.append('localindus')
    if row['dependance']:
        categories.append('dependance')
    return ' '.join(categories)


df_preformatted['property_type_category'] = df_preformatted.apply(create_combined_property_type, axis=1)

df_preformatted['property_type_category'].unique()

# TRANSFORMER LE PROPERTY TYPE CATEGORIES POUR FACILITER LA VISU SUR LES COURBES
dummies = pd.get_dummies(df_preformatted['property_type_category'], prefix='property_type', dtype=int)

# Concaténer les colonnes dummies avec le DataFrame original
df_preformatted = pd.concat([df_preformatted, dummies], axis=1)

#### colonne qui calcul les tranches de valeurs foncieres

In [7]:
def create_combined_val_category(row):
    val_categories = ""
    if row['valeur_fonciere'] <= 4999:
        val_categories = "0 - 4 999"
    elif row['valeur_fonciere'] >= 5000 and row['valeur_fonciere'] <= 49999:
        val_categories = "5 000 - 49 999"
    elif row['valeur_fonciere'] >= 50000 and row['valeur_fonciere'] <= 99999:
        val_categories = "50 000 - 100 000"
    elif row['valeur_fonciere'] >= 100000 and row['valeur_fonciere'] <= 199999:
        val_categories = "100 000 - 199 999"
    elif row['valeur_fonciere'] >= 200000 and row['valeur_fonciere'] <= 299999:
        val_categories = "200 000 - 299 999"
    elif row['valeur_fonciere'] >= 300000 and row['valeur_fonciere'] <= 399999:
        val_categories = "300 000 - 399 999"
    elif row['valeur_fonciere'] >= 400000 and row['valeur_fonciere'] <= 499999:
        val_categories = "400 000 - 499 999"
    elif row['valeur_fonciere'] >= 500000 and row['valeur_fonciere'] <= 749999:
        val_categories = "500 000 - 749 999"
    elif row['valeur_fonciere'] >= 750000 and row['valeur_fonciere'] <= 999999:
        val_categories = "750 000 - 999 999"
    elif row['valeur_fonciere'] >= 1000000 and row['valeur_fonciere'] <= 4999999:
        val_categories = "1 000 000 - 4 999 999"
    elif row['valeur_fonciere'] >= 5000000:
        val_categories = "5 000 000 et +"
    else:
        val_categories = "valeur non défini"
    return val_categories


df_preformatted['val_fonciere_category'] = df_preformatted.apply(create_combined_val_category, axis=1)

df_preformatted['val_fonciere_category'].unique()

array(['500 000 - 749 999', '200 000 - 299 999', '400 000 - 499 999',
       '0 - 4 999', '1 000 000 - 4 999 999', '300 000 - 399 999',
       '50 000 - 100 000', '100 000 - 199 999', '750 000 - 999 999',
       '5 000 - 49 999', '5 000 000 et +'], dtype=object)

In [8]:
df_preformatted.head(20)

,date_mutation,valeur_fonciere,code_voie,code_postal,code_commune,no_plan,nb_lot,surface_reel_bati,nb_piece_prin,surf_terrain,...,property_type_appartement maison localindus,property_type_appartement maison localindus dependance,property_type_dependance,property_type_localindus,property_type_localindus dependance,property_type_maison,property_type_maison dependance,property_type_maison localindus,property_type_maison localindus dependance,val_fonciere_category
0,2023-11-29,553500,0811,31770,149,62,9,67,3,0,...,0,0,0,0,0,0,0,0,0,500 000 - 749 999
1,2023-11-15,286110,B058,31400,555,41,0,0,0,9537,...,0,0,0,0,0,0,0,0,0,200 000 - 299 999
2,2023-11-15,406414,B055,31590,573,2824,0,72,3,2217,...,0,0,0,0,0,1,0,0,0,400 000 - 499 999
3,2023-11-15,1,B053,31790,490,26,0,0,0,1120,...,0,0,0,0,0,0,0,0,0,0 - 4 999
4,2023-11-22,1,1580,31600,395,324,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0 - 4 999
5,2023-11-29,424000,0802,31770,149,32,7,63,3,0,...,0,0,0,0,0,0,0,0,0,400 000 - 499 999
6,2023-11-15,1954000,B063,31300,555,91,3,0,0,65235,...,0,0,0,0,0,0,0,0,0,1 000 000 - 4 999 999
7,2023-11-23,330000,2975,31200,555,346,2,49,2,0,...,0,0,0,0,0,0,0,0,0,300 000 - 399 999
8,2023-11-27,312480,0693,31120,433,60,2,51,2,0,...,0,0,0,0,0,0,0,0,0,300 000 - 399 999
9,2023-11-15,65500,B032,31340,73,121,0,0,0,1481,...,0,0,0,0,0,0,0,0,0,50 000 - 100 000


### ajout de la population par commune et par an

#### df population 1876 2021

In [10]:
file_path_1876_2021 = r"C:\Users\gaeta\Documents\YNOV\master1\open_data\visualisation\population\base-pop-historiques-1876-2021 1.xlsx"
df_1876_2021 = pd.read_excel(file_path_1876_2021, header=5)

df_1876_2021 = df_1876_2021[['CODGEO', 'DEP', 'PMUN2021', 'PMUN2020', 'PMUN2019', 'PMUN2018', 'PMUN2017', 'PMUN2012', 'PTOT1921']]
df_1876_2021.dtypes

CODGEO       object
DEP          object
PMUN2021      int64
PMUN2020      int64
PMUN2019      int64
PMUN2018      int64
PMUN2017      int64
PMUN2012      int64
PTOT1921    float64
dtype: object

In [11]:
df_1876_2021 = df_1876_2021[df_1876_2021['DEP'] == '31']
df_1876_2021.head()

,CODGEO,DEP,PMUN2021,PMUN2020,PMUN2019,PMUN2018,PMUN2017,PMUN2012,PTOT1921
11251,31001,31,119,120,122,121,118,114,285.0
11252,31002,31,239,242,241,243,245,243,610.0
11253,31003,31,1280,1271,1272,1274,1275,1135,93.0
11254,31004,31,2764,2721,2679,2663,2655,2440,437.0
11255,31005,31,287,291,293,296,305,316,701.0


In [12]:
# df_1876_2021 = df_1876_2021[~df_1876_2021['CODGEO'].isin(['2A001', '2A004', '2A006', '2A008', '2A011'])]

df_1876_2021 = df_1876_2021.drop(columns=['DEP'])
df_1876_2021['CODGEO'] = df_1876_2021['CODGEO'].astype(str).str[2:].fillna(0).astype('int64')
df_1876_2021['PTOT1921'] = df_1876_2021['PTOT1921'].fillna(0).astype('int64')

df_1876_2021 = df_1876_2021.rename(columns={'CODGEO': 'code_commune',
                                           'PMUN2021': 'population_2021',
                                           'PMUN2020': 'population_2020',
                                           'PMUN2019': 'population_2019',
                                           'PMUN2018':'population_2018',
                                           'PMUN2017':'population_2017',
                                           'PMUN2012':'population_2012',
                                           'PTOT1921':'population_1921'})

df_1876_2021.reset_index(drop=True, inplace=True)
df_1876_2021.dtypes

code_commune       int64
population_2021    int64
population_2020    int64
population_2019    int64
population_2018    int64
population_2017    int64
population_2012    int64
population_1921    int64
dtype: object

In [13]:
df_1876_2021.head()

,code_commune,population_2021,population_2020,population_2019,population_2018,population_2017,population_2012,population_1921
0,1,119,120,122,121,118,114,285
1,2,239,242,241,243,245,243,610
2,3,1280,1271,1272,1274,1275,1135,93
3,4,2764,2721,2679,2663,2655,2440,437
4,5,287,291,293,296,305,316,701


In [14]:
df_1876_2021['code_commune']

0        1
1        2
2        3
3        4
4        5
      ... 
581    589
582    590
583    591
584    592
585    593
Name: code_commune, Length: 586, dtype: int64

#### df population 2022

In [15]:
file_path_2022 = r"C:\Users\gaeta\Documents\YNOV\master1\open_data\visualisation\population\demographie2022.xlsx"
df_2022 = pd.read_excel(file_path_2022, sheet_name='Communes', header=7)

df_2022 = df_2022[['Code commune', 'Population totale']]

df_2022 = df_2022.rename(columns={'Code commune': 'code_commune', 'Population totale':'population_2022'})

df_2022.dtypes

code_commune       int64
population_2022    int64
dtype: object

In [16]:
df_2022.head()

,code_commune,population_2022
0,1,125
1,2,247
2,3,1301
3,5,301
4,6,227


In [17]:
df_2022['code_commune']

0        1
1        2
2        3
3        5
4        6
      ... 
581    585
582    586
583    587
584    588
585    589
Name: code_commune, Length: 586, dtype: int64

##### merge population 1876_2021 et population 2022 sur le code_commune

In [18]:
df_1876_2022 = pd.merge(df_2022, df_1876_2021, on='code_commune')

df_1876_2022.head()

,code_commune,population_2022,population_2021,population_2020,population_2019,population_2018,population_2017,population_2012,population_1921
0,1,125,119,120,122,121,118,114,285
1,2,247,239,242,241,243,245,243,610
2,3,1301,1280,1271,1272,1274,1275,1135,93
3,5,301,287,291,293,296,305,316,701
4,6,227,228,226,221,215,210,198,163


##### calcul des pourcentages d'evolutions

In [19]:
df_1876_2022['%_evolution_10ans_2012_2022'] = ((df_1876_2022['population_2022'] - df_1876_2022['population_2012']) / df_1876_2022['population_2012']) * 100
df_1876_2022['%_evolution_10ans_2012_2022'] = df_1876_2022['%_evolution_10ans_2012_2022'].astype('int64')
df_1876_2022['evolution_10ans_2012_2022'] = df_1876_2022['population_2022'] - df_1876_2022['population_2012']

df_1876_2022['%_evolution_100ans_1921_2021'] = ((df_1876_2022['population_2021'] - df_1876_2022['population_1921']) / df_1876_2022['population_1921']) * 100
df_1876_2022['%_evolution_100ans_1921_2021'] = df_1876_2022['%_evolution_100ans_1921_2021'].astype('int64')
df_1876_2022['evolution_100ans_1921_2021'] = df_1876_2022['population_2021'] - df_1876_2022['population_1921']

df_1876_2022.head()

,code_commune,population_2022,population_2021,population_2020,population_2019,population_2018,population_2017,population_2012,population_1921,%_evolution_10ans_2012_2022,evolution_10ans_2012_2022,%_evolution_100ans_1921_2021,evolution_100ans_1921_2021
0,1,125,119,120,122,121,118,114,285,9,11,-58,-166
1,2,247,239,242,241,243,245,243,610,1,4,-60,-371
2,3,1301,1280,1271,1272,1274,1275,1135,93,14,166,1276,1187
3,5,301,287,291,293,296,305,316,701,-4,-15,-59,-414
4,6,227,228,226,221,215,210,198,163,14,29,39,65


In [20]:
import os

output_path = r"C:\Users\gaeta\Documents\YNOV\master1\open_data\visualisation"
output_file = "df_ti_pe_1876_2022.csv"

df_1876_2022.to_csv(os.path.join(output_path, output_file), index=False)

##### merge du df de base avec les populations et pourcentage 

In [21]:
df_preformatted = pd.merge(df_preformatted, df_1876_2022, on='code_commune')

df_preformatted.head()

,date_mutation,valeur_fonciere,code_voie,code_postal,code_commune,no_plan,nb_lot,surface_reel_bati,nb_piece_prin,surf_terrain,...,population_2020,population_2019,population_2018,population_2017,population_2012,population_1921,%_evolution_10ans_2012_2022,evolution_10ans_2012_2022,%_evolution_100ans_1921_2021,evolution_100ans_1921_2021
0,2023-11-29,553500,0811,31770,149,62,9,67,3,0,...,39866,39968,39097,38951,36699,1556,10,3830,2480,38603
1,2023-11-29,424000,0802,31770,149,32,7,63,3,0,...,39866,39968,39097,38951,36699,1556,10,3830,2480,38603
2,2023-11-17,1,B244,31770,149,111,0,0,0,604,...,39866,39968,39097,38951,36699,1556,10,3830,2480,38603
3,2023-11-29,825700,0712,31770,149,814,0,110,4,834,...,39866,39968,39097,38951,36699,1556,10,3830,2480,38603
4,2023-11-29,320800,0714,31770,149,812,0,101,5,241,...,39866,39968,39097,38951,36699,1556,10,3830,2480,38603


In [22]:
df_preformatted['date_mutation'] = pd.to_datetime(df_preformatted['date_mutation'])

# Trier le DataFrame par la colonne 'date_mutation'
df_preformatted = df_preformatted.sort_values(by='date_mutation')

In [25]:
import os

output_path = r"C:\Users\gaeta\Documents\YNOV\master1\open_data\visualisation"
output_file = "df_ti_pe_complet_ALL_DONE.csv"

df_preformatted.to_csv(os.path.join(output_path, output_file), index=False)